### Cargue de datos inicial y generación de rutas para procesamiento de imágenes

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ruta_fuente = '/content/drive/MyDrive/archivos_proyecto_deep/'

In [ ]:
datos_pautas['Oferta'] = datos_pautas['Oferta'].str.split('-').str[0]

In [ ]:
excel_datos = 'fuentes.xlsx'
ruta_excel =ruta_fuente+excel_datos

print(ruta_excel)
datos_pautas = pd.read_excel(ruta_excel, sheet_name="Datos 1")

/content/drive/MyDrive/archivos_proyecto_deep/fuentes.xlsx


In [ ]:
datos_pautas.columns

Index(['Fecha', 'industria', 'marca', 'Anunciante', 'Producto', 'Gatillador',
       'Oferta', 'Nombre de campaña', 'Landing page de la campaña',
       'Etiquetas de campaña', 'Sitio web', 'Secciones del sitio web',
       'Editor', 'Formato', 'Tamaño de aviso', 'Duración (Video)',
       'Omitible (Video)', 'Posición', 'Advertisement', 'Screenshot',
       'Country', 'Device', 'Hospedado por', 'Vendido por', 'Impacto',
       'Impresiones', 'Valorización (Local)', 'Valorización (Dólares)',
       'Ads Count', 'Reporte web'],
      dtype='object')

In [ ]:
datos_seleccionados = datos_pautas[['marca', 'Producto', 'Advertisement', 'Gatillador', 'Oferta']][datos_pautas['Advertisement'].str.contains('.jpg')].head(6000)
print(datos_seleccionados.columns)
print(datos_seleccionados.shape)
datos_seleccionados.head(5)

Index(['marca', 'Producto', 'Advertisement', 'Gatillador', 'Oferta'], dtype='object')
(6000, 5)


,marca,Producto,Advertisement,Gatillador,Oferta
0,claro,MOVIL,https://ads.admetricks.com/banner_a277eaa2a4aa...,EQUIPO,Equipos
92,entel,PREPAGO,https://ads.admetricks.com/banner_f87fe889bce9...,SERVICIO,Prepago
93,entel,PREPAGO,https://ads.admetricks.com/banner_e1fdbec912b5...,SERVICIO,Prepago
94,entel,PREPAGO,https://ads.admetricks.com/banner_a91ac79a5189...,SERVICIO,Prepago
95,entel,PREPAGO,https://ads.admetricks.com/banner_843052611b33...,SERVICIO,Prepago


In [ ]:
datos_seleccionados = datos_seleccionados.drop_duplicates()
datos_seleccionados.shape

(4862, 4)

In [ ]:
# Cargar el dataframe
df = datos_seleccionados


# Recorrer cada fila del DataFrame
for index, row in df.iterrows():
  # Obtener el ID de imagen (asumiendo que está en la columna 'ID_Imagen')
  id_imagen = index
  operador = row['marca']

  # Construir la ruta local de la imagen usando el ID como nombre de archivo
  ruta_imagen = ruta_fuente+operador+'/'+str(id_imagen)+'.jpg'  # Ajustar el tipo de archivo

  # Agregar la ruta local de la imagen a la fila
  df.loc[index, 'ruta_local_imagen'] = ruta_imagen


# Guardar el dataframe
nombre_dataframe_rutas = 'dataframe_rutas.pkl'
ruta_dataframe_rutas = ruta_fuente+nombre_dataframe_rutas
print(ruta_dataframe_rutas)
df.to_pickle(ruta_dataframe_rutas)

# Imprimir el DataFrame actualizado
df.head(6)

/content/drive/MyDrive/archivos_proyecto_deep/dataframe_rutas.pkl


,marca,Producto,Advertisement,Gatillador,ruta_local_imagen
0,claro,MOVIL,https://ads.admetricks.com/banner_a277eaa2a4aa...,EQUIPO,/content/drive/MyDrive/archivos_proyecto_deep/...
92,entel,PREPAGO,https://ads.admetricks.com/banner_f87fe889bce9...,SERVICIO,/content/drive/MyDrive/archivos_proyecto_deep/...
93,entel,PREPAGO,https://ads.admetricks.com/banner_e1fdbec912b5...,SERVICIO,/content/drive/MyDrive/archivos_proyecto_deep/...
94,entel,PREPAGO,https://ads.admetricks.com/banner_a91ac79a5189...,SERVICIO,/content/drive/MyDrive/archivos_proyecto_deep/...
95,entel,PREPAGO,https://ads.admetricks.com/banner_843052611b33...,SERVICIO,/content/drive/MyDrive/archivos_proyecto_deep/...
96,entel,PREPAGO,https://ads.admetricks.com/banner_7fc0aed40440...,SERVICIO,/content/drive/MyDrive/archivos_proyecto_deep/...


In [ ]:
import pickle

with open('/content/drive/MyDrive/archivos_proyecto_deep/dataframe_rutas.pkl', 'rb') as f:
    df = pickle.load(f)

df.head()

,marca,Producto,Advertisement,Gatillador,ruta_local_imagen
0,claro,MOVIL,https://ads.admetricks.com/banner_a277eaa2a4aa...,EQUIPO,/content/drive/MyDrive/archivos_proyecto_deep/...
92,entel,PREPAGO,https://ads.admetricks.com/banner_f87fe889bce9...,SERVICIO,/content/drive/MyDrive/archivos_proyecto_deep/...
93,entel,PREPAGO,https://ads.admetricks.com/banner_e1fdbec912b5...,SERVICIO,/content/drive/MyDrive/archivos_proyecto_deep/...
94,entel,PREPAGO,https://ads.admetricks.com/banner_a91ac79a5189...,SERVICIO,/content/drive/MyDrive/archivos_proyecto_deep/...
95,entel,PREPAGO,https://ads.admetricks.com/banner_843052611b33...,SERVICIO,/content/drive/MyDrive/archivos_proyecto_deep/...


## Descargar imágenes y alamacenarlas en drive

In [ ]:
import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from tqdm import tqdm

# Crear los directorios necesarios
def create_directories(base_path, brands):
    for brand in brands:
        os.makedirs(os.path.join(base_path, brand), exist_ok=True)

# Función para descargar y guardar imágenes
def download_image(url, save_path):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))
        img.save(save_path)
    except Exception as e:
        print(f"Failed to download {url}: {e}")


In [ ]:
# Obtener las marcas únicas para crear directorios
brands = df['marca'].unique()
#create_directories(ruta_fuente, brands)


In [ ]:
# Descargar y guardar las imágenes en los directorios correspondientes
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    brand = row['marca']
    image_url = row['Advertisement']
    save_path = os.path.join(ruta_fuente, brand, f"{index}.jpg")
    download_image(image_url, save_path)

 30%|██▉       | 1448/4862 [21:23<3:58:31,  4.19s/it]

Failed to download https://ads.admetricks.com/banner_d2ddfc7029a1490328be977ccaf2ab35.jpg: 520 Server Error:  for url: https://ads.admetricks.com/banner_d2ddfc7029a1490328be977ccaf2ab35.jpg


100%|██████████| 4862/4862 [1:13:10<00:00,  1.11it/s]


### Redimensionar y Normalizar Imagenes

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Definir un generador de datos con aumentación de datos
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Usar el 20% de los datos para validación
)

# Configurar generadores para entrenamiento y validación
train_generator = datagen.flow_from_directory(
    base_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 4022 images belonging to 6 classes.
Found 1001 images belonging to 6 classes.


### Crear el Modelo con Transfer Learning

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam

# Cargar el modelo base preentrenado VGG16 sin la capa superior
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Añadir nuevas capas para la clasificación
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output_empresa = Dense(len(brands), activation='softmax', name='empresa_output')(x)

# Crear el modelo final
model = Model(inputs=base_model.input, outputs=output_empresa)

# Compilar el modelo
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
# Entrenar el modelo
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)



Epoch 1/10
125/125 [==============================] - 121s 725ms/step - loss: 0.5635 - accuracy: 0.8143 - val_loss: 0.8704 - val_accuracy: 0.6633
Epoch 2/10
125/125 [==============================] - 80s 638ms/step - loss: 0.2987 - accuracy: 0.9120 - val_loss: 0.9248 - val_accuracy: 0.7228
Epoch 3/10
125/125 [==============================] - 85s 677ms/step - loss: 0.2367 - accuracy: 0.9301 - val_loss: 0.9033 - val_accuracy: 0.6663
Epoch 4/10
125/125 [==============================] - 84s 675ms/step - loss: 0.2127 - accuracy: 0.9348 - val_loss: 0.9012 - val_accuracy: 0.6885
Epoch 5/10
125/125 [==============================] - 86s 691ms/step - loss: 0.1899 - accuracy: 0.9401 - val_loss: 0.9981 - val_accuracy: 0.6190
Epoch 6/10
125/125 [==============================] - 83s 665ms/step - loss: 0.1772 - accuracy: 0.9446 - val_loss: 0.8677 - val_accuracy: 0.6663
Epoch 7/10
125/125 [==============================] - 81s 645ms/step - loss: 0.1622 - accuracy: 0.9484 - val_loss: 0.8564 - val_a

In [ ]:
model
print(brands)

['claro' 'entel' 'movistar' 'bantel perú' 'bitel' 'directv']


In [ ]:
salida_modelo = 'model_empresa_classification.h5'
ruta_modelo = base_path+salida_modelo
print(ruta_modelo)

# Guardar el modelo entrenado
model.save(ruta_modelo)

/content/drive/MyDrive/archivos_proyecto_deep/model_empresa_classification.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Clasificar el Gatillador


In [ ]:
# Añadir capas adicionales para la clasificación del gatillador
output_gatillador = Dense(len(df['Gatillador'].unique()), activation='softmax', name='gatillador_output')(x)

# Crear el modelo con dos salidas
model_multi_output = Model(inputs=base_model.input, outputs=[output_empresa, output_gatillador])

# Compilar el modelo con dos pérdidas
model_multi_output.compile(optimizer=Adam(lr=0.001),
                           loss={'empresa_output': 'categorical_crossentropy', 'gatillador_output': 'categorical_crossentropy'},
                           metrics=['accuracy'])

# Resumen del modelo
model_multi_output.summary()

# Entrenar el modelo con dos salidas
model_multi_output.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# Guardar el modelo entrenado
model_multi_output.save('model_empresa_gatillador_classification.h5')

In [ ]:
# prompt: print hola

print("hola")
brands


hola


array(['claro', 'entel', 'movistar', 'bantel perú', 'bitel', 'directv'],
      dtype=object)

## Nueva prueba extrayendo texto de la imagen..

In [ ]:
!pip install pytesseract

In [ ]:
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,417 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
from tensorflow.keras.preprocessing import image
import pytesseract
from PIL import Image

def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

# Función para extraer texto de una imagen utilizando OCR
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

extract_text_from_image('/content/drive/MyDrive/archivos_proyecto_deep/directv/20542.jpg')


'Llego la Fibra a DIRECTV Ox\n\nContrata DGO + Fibra + Ligaimax por solo $/149 por\nmes. Te duplicamos los megas!\n\x0c'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input, Embedding, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pytesseract
from PIL import Image
import os
import re
import nltk
from sklearn.model_selection import train_test_split

# Descargar recursos necesarios de nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Asegúrate de que pytesseract apunte al ejecutable de Tesseract
#pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('spanish'))

# Limpieza de texto
def clean_text(text):
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text)
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text

# Extraer texto de imaganes
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

#Procesar imaganes
def process_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    x = image.img_to_array(img)
    #x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [ ]:
process_image('/content/drive/MyDrive/archivos_proyecto_deep/claro/291.jpg').shape

(224, 224, 3)

In [ ]:
text = extract_text_from_image('/content/drive/MyDrive/archivos_proyecto_deep/claro/291.jpg')
print(text)
cleaned_text = clean_text(text)
print(cleaned_text)

Claro Perd @
Sponsored - @

iSol, playa, arena y la Fibra Optica de Claro en tu casa... See more

Claro~

  

Nera}

GRATIS

tienda
Internet TODO el verano! [ SHOP NOW |
En el mar la vida es més sabrosa si contratas la... —————

 

aik

claro perd sponsored isol playa arena fibra optica claro casa see more claro nera gratis tienda internet verano shop now mar vida més sabrosa si contratas aik


In [ ]:
max_words = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words)

In [ ]:
for operador in brands:
  print(operador)

claro
entel
movistar
bantel perú
bitel
directv


In [ ]:
from sklearn.preprocessing import LabelEncoder
# Obtener las rutas de las imágenes y las etiquetas de los operadores del dataframe
all_image_paths = df['ruta_local_imagen'].tolist()
labels = df['marca'].tolist()

# Inicializar listas para almacenar características de imágenes y textos limpios
image_features_list = []
cleaned_texts = []

# Procesar cada imagen y extraer texto
for image_path in all_image_paths:
    # Extraer características visuales de la imagen
    image_features = process_image(image_path)

    # Extraer y limpiar texto de la imagen
    text = extract_text_from_image(image_path)
    cleaned_text = clean_text(text)

    # Agregar características visuales y texto limpio a las listas
    image_features_list.append(image_features)
    cleaned_texts.append(cleaned_text)

# Convertir listas a arrays numpy
image_features_array = np.array(image_features_list)
text_sequences = tokenizer.texts_to_sequences(cleaned_texts)
text_data = pad_sequences(text_sequences, maxlen=max_sequence_length)

# Convertir las etiquetas de texto a enteros
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Convertir las etiquetas a formato categórico
labels_categorical = to_categorical(labels_encoded)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_image, X_test_image, X_train_text, X_test_text, y_train, y_test = train_test_split(
    image_features_array, text_data, labels_categorical, test_size=0.2, random_state=42)


In [ ]:
# Verificar el mapeo de etiquetas
for index, label in enumerate(label_encoder.classes_):
    print(f"{index}: {label}")

0: bantel perú
1: bitel
2: claro
3: directv
4: entel
5: movistar


In [ ]:
import pickle

# Save split data in pickle format
with open('/content/drive/MyDrive/archivos_proyecto_deep/image_features_train.pkl', 'wb') as f:
    pickle.dump(X_train_image, f)
with open('/content/drive/MyDrive/archivos_proyecto_deep/image_features_test.pkl', 'wb') as f:
    pickle.dump(X_test_image, f)
with open('/content/drive/MyDrive/archivos_proyecto_deep/text_data_train.pkl', 'wb') as f:
    pickle.dump(X_train_text, f)
with open('/content/drive/MyDrive/archivos_proyecto_deep/text_data_test.pkl', 'wb') as f:
    pickle.dump(X_test_text, f)
with open('/content/drive/MyDrive/archivos_proyecto_deep/labels_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)
with open('/content/drive/MyDrive/archivos_proyecto_deep/labels_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)

print("Data saved successfully!")

Data saved successfully!


In [ ]:
import pickle

# Load DataFrames from pickle files
with open('/content/drive/MyDrive/archivos_proyecto_deep/image_features_train.pkl', 'rb') as f:
    X_train_image = pickle.load(f)

with open('/content/drive/MyDrive/archivos_proyecto_deep/image_features_test.pkl', 'rb') as f:
    X_test_image = pickle.load(f)

with open('/content/drive/MyDrive/archivos_proyecto_deep/text_data_train.pkl', 'rb') as f:
    X_train_text = pickle.load(f)

with open('/content/drive/MyDrive/archivos_proyecto_deep/text_data_test.pkl', 'rb') as f:
    X_test_text = pickle.load(f)

with open('/content/drive/MyDrive/archivos_proyecto_deep/labels_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('/content/drive/MyDrive/archivos_proyecto_deep/labels_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

print("Data loaded successfully!")

# Verificar las formas de los datos
print(X_train_image.shape, X_test_image.shape)
print(X_train_text.shape, X_test_text.shape)
print(y_train.shape, y_test.shape)

Data loaded successfully!
(3889, 224, 224, 3) (973, 224, 224, 3)
(3889, 100) (973, 100)
(3889, 6) (973, 6)


In [ ]:
from tensorflow.keras.optimizers import Adam

# Cargar el modelo VGG16 preentrenado sin la última capa
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Definir las entradas
num_classes = len(brands)
image_input = base_model.input
text_input = Input(shape=(max_sequence_length,), dtype='int32', name='text_input')

# Procesar la entrada de imagen
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)

# Procesar la entrada de texto
text_embedding = Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length)(text_input)
text_lstm = LSTM(128)(text_embedding)

# Concatenar las dos salidas
concatenated = Concatenate()([x, text_lstm])
output = Dense(num_classes, activation='softmax')(concatenated)

# Definir el modelo
model = Model(inputs=[image_input, text_input], outputs=output)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit([X_train_image, X_train_text], y_train, validation_data=([X_test_image, X_test_text], y_test), epochs=10, batch_size=32)

Epoch 1/10
122/122 [==============================] - 42s 255ms/step - loss: 0.6095 - accuracy: 0.8246 - val_loss: 0.2525 - val_accuracy: 0.9291
Epoch 2/10
122/122 [==============================] - 19s 155ms/step - loss: 0.2415 - accuracy: 0.9306 - val_loss: 0.1939 - val_accuracy: 0.9414
Epoch 3/10
122/122 [==============================] - 21s 170ms/step - loss: 0.1756 - accuracy: 0.9457 - val_loss: 0.1879 - val_accuracy: 0.9445
Epoch 4/10
122/122 [==============================] - 21s 172ms/step - loss: 0.1397 - accuracy: 0.9576 - val_loss: 0.1659 - val_accuracy: 0.9517
Epoch 5/10
122/122 [==============================] - 21s 174ms/step - loss: 0.1182 - accuracy: 0.9635 - val_loss: 0.1564 - val_accuracy: 0.9527
Epoch 6/10
122/122 [==============================] - 21s 175ms/step - loss: 0.0985 - accuracy: 0.9704 - val_loss: 0.1571 - val_accuracy: 0.9507
Epoch 7/10
122/122 [==============================] - 21s 169ms/step - loss: 0.0865 - accuracy: 0.9743 - val_loss: 0.1441 - val_ac

In [ ]:
salida_modelo = '/content/drive/MyDrive/archivos_proyecto_deep/model_empresa_classification_compuesto.h5'
print(salida_modelo)

# Guardar el modelo entrenado
model.save(salida_modelo)

/content/drive/MyDrive/archivos_proyecto_deep/model_empresa_classification_compuesto.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Marca y Gatillador

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input, Embedding, LSTM, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pytesseract
from PIL import Image
import os
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Descargar recursos necesarios de nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('spanish'))

# Función para limpiar texto
def clean_text(text):
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text)
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text

# Función para extraer texto de una imagen
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

# Función para procesar una imagen
def process_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [ ]:
# Suponiendo que df es tu DataFrame con las columnas 'marca', 'gatillador', y 'ruta_local_imagen'
all_image_paths = df['ruta_local_imagen'].tolist()
labels_marca = df['marca'].tolist()
labels_gatillador = df['Gatillador'].tolist()

# Inicializar listas para almacenar características de imágenes y textos limpios
image_features_list = []
cleaned_texts = []

# Procesar cada imagen y extraer texto
for image_path in all_image_paths:
    # Extraer características visuales de la imagen
    image_features = process_image(image_path)

    # Extraer y limpiar texto de la imagen
    text = extract_text_from_image(image_path)
    cleaned_text = clean_text(text)

    # Agregar características visuales y texto limpio a las listas
    image_features_list.append(image_features)
    cleaned_texts.append(cleaned_text)

# Convertir listas a arrays numpy
image_features_array = np.array(image_features_list).squeeze()  # Quitar dimensión extra



In [ ]:
from sklearn.preprocessing import LabelEncoder

max_words = 10000
max_sequence_length = 100

# Configurar el tokenizer
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(cleaned_texts)
text_sequences = tokenizer.texts_to_sequences(cleaned_texts)
text_data = pad_sequences(text_sequences, maxlen=max_sequence_length)

# Codificar las etiquetas de texto a enteros
label_encoder_marca = LabelEncoder()
labels_marca_encoded = label_encoder_marca.fit_transform(labels_marca)

label_encoder_gatillador = LabelEncoder()
labels_gatillador_encoded = label_encoder_gatillador.fit_transform(labels_gatillador)

# Convertir las etiquetas a formato categórico
labels_marca_categorical = to_categorical(labels_marca_encoded)
labels_gatillador_categorical = to_categorical(labels_gatillador_encoded)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_image, X_test_image, X_train_text, X_test_text, y_train_marca, y_test_marca, y_train_gatillador, y_test_gatillador = train_test_split(
    image_features_array, text_data, labels_marca_categorical, labels_gatillador_categorical, test_size=0.2, random_state=42)

In [ ]:
import pickle

# Guardar los label encoders en archivos pickle
with open('/content/drive/MyDrive/archivos_proyecto_deep/label_encoder_marca.pkl', 'wb') as f:
    pickle.dump(label_encoder_marca, f)

with open('/content/drive/MyDrive/archivos_proyecto_deep/label_encoder_gatillador.pkl', 'wb') as f:
    pickle.dump(label_encoder_gatillador, f)

print("Label encoders saved successfully!")

# Guardar las listas en archivos pickle
with open('/content/drive/MyDrive/archivos_proyecto_deep/image_features_list.pkl', 'wb') as f:
    pickle.dump(image_features_list, f)

with open('/content/drive/MyDrive/archivos_proyecto_deep/cleaned_texts.pkl', 'wb') as f:
    pickle.dump(cleaned_texts, f)

print("Lists saved successfully!")


Label encoders saved successfully!
Lists saved successfully!


In [ ]:
import pickle

# Save split data in pickle format
with open('/content/drive/MyDrive/archivos_proyecto_deep/image_features_train_gatillador.pkl', 'wb') as f:
    pickle.dump(X_train_image, f)
with open('/content/drive/MyDrive/archivos_proyecto_deep/image_features_train_gatillador.pkl', 'wb') as f:
    pickle.dump(X_test_image, f)
with open('/content/drive/MyDrive/archivos_proyecto_deep/image_features_train_gatillador.pkl', 'wb') as f:
    pickle.dump(X_train_text, f)
with open('/content/drive/MyDrive/archivos_proyecto_deep/image_features_train_gatillador.pkl', 'wb') as f:
    pickle.dump(X_test_text, f)
with open('/content/drive/MyDrive/archivos_proyecto_deep/y_train_marca_gatillador.pkl', 'wb') as f:
    pickle.dump(y_train_marca, f)
with open('/content/drive/MyDrive/archivos_proyecto_deep/y_test_marca_gatillador.pkl', 'wb') as f:
    pickle.dump(y_test_marca, f)
with open('/content/drive/MyDrive/archivos_proyecto_deep/y_train_gatillador.pkl', 'wb') as f:
    pickle.dump(y_train_gatillador, f)
with open('/content/drive/MyDrive/archivos_proyecto_deep/y_test_gatillador.pkl', 'wb') as f:
    pickle.dump(y_test_gatillador, f)

print("Data saved successfully!")

Data saved successfully!


In [ ]:
# Verificar las formas de los datos
print(X_train_image.shape, X_test_image.shape)
print(X_train_text.shape, X_test_text.shape)
print(y_train_marca.shape, y_test_marca.shape)
print(y_train_gatillador.shape, y_test_gatillador.shape)

(3889, 224, 224, 3) (973, 224, 224, 3)
(3889, 100) (973, 100)
(3889, 6) (973, 6)
(3889, 27) (973, 27)


In [ ]:
# Verificar el mapeo de etiquetas
for index, label in enumerate(label_encoder_marca.classes_):
    print(f"{index}: {label}")

0: bantel perú
1: bitel
2: claro
3: directv
4: entel
5: movistar


In [ ]:
# Verificar el mapeo de etiquetas
for index, label in enumerate(label_encoder_gatillador.classes_):
    print(f"{index}: {label}")

0: ACCESORIO
1: CHIP
2: COMBO
3: COMUNICACION
4: CONCURSO
5: CUOTAS
6: CURSOS
7: DATOS 5G
8: DESCARGA
9: DESCUENTOS 
10: ENTRENIMIENTO
11: ENVIO GRATIS
12: EQUIPO
13: EQUIPO+SERVICIO
14: FUTBOL
15: ILIMITADO
16: INSTALACION GRATIS
17: MAS GIGAS
18: MESES GRATIS
19: PLANES ILIMITADOS
20: PORTABILIDAD
21: RRSS ILIMITADAS
22: SERVICIO
23: SERVICIO GRATIS
24: SIN CUOTA INICIAL 
25: STREAMING
26: VELOCIDAD


In [ ]:
df['Gatillador'].unique()

array(['EQUIPO', 'SERVICIO', 'DATOS 5G', 'INSTALACION GRATIS',
       'ENVIO GRATIS', 'DESCUENTOS ', 'SIN CUOTA INICIAL ', 'ACCESORIO',
       'CUOTAS', 'VELOCIDAD', 'COMBO', 'STREAMING', 'COMUNICACION',
       'ILIMITADO', 'MESES GRATIS', 'FUTBOL', 'SERVICIO GRATIS',
       'RRSS ILIMITADAS', 'MAS GIGAS', 'PORTABILIDAD', 'CURSOS',
       'PLANES ILIMITADOS', 'CHIP', 'DESCARGA', 'EQUIPO+SERVICIO',
       'ENTRENIMIENTO', 'CONCURSO'], dtype=object)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input, Embedding, LSTM, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from PIL import Image
import os
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Descargar recursos necesarios de nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
# Function to load data from pickle files
def load_data(pickle_file_path):
    with open(pickle_file_path, 'rb') as f:
        data = pickle.load(f)
    return datastop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
tokenizer

In [ ]:
# Function to load data from pickle files
def load_data(pickle_file_path):
    with open(pickle_file_path, 'rb') as f:
        data = pickle.load(f)
    return data

image_features_list = load_data('/content/drive/MyDrive/archivos_proyecto_deep/image_features_list.pkl')
cleaned_texts = load_data('/content/drive/MyDrive/archivos_proyecto_deep/cleaned_texts.pkl')



In [ ]:
image_features_array = np.array(image_features_list).squeeze()  # Quitar dimensión extra
image_features_array.shape

(4862, 224, 224, 3)

In [ ]:
max_words = 10000
max_sequence_length = 100
all_image_paths = df['ruta_local_imagen'].tolist()
labels_marca = df['marca'].tolist()
labels_gatillador = df['Gatillador'].tolist()

# Configurar el tokenizer
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(cleaned_texts)
text_sequences = tokenizer.texts_to_sequences(cleaned_texts)
text_data = pad_sequences(text_sequences, maxlen=max_sequence_length)

# Codificar las etiquetas de texto a enteros
label_encoder_marca = LabelEncoder()
labels_marca_encoded = label_encoder_marca.fit_transform(labels_marca)

label_encoder_gatillador = LabelEncoder()
labels_gatillador_encoded = label_encoder_gatillador.fit_transform(labels_gatillador)

# Convertir las etiquetas a formato categórico
labels_marca_categorical = to_categorical(labels_marca_encoded)
labels_gatillador_categorical = to_categorical(labels_gatillador_encoded)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_image, X_test_image, X_train_text, X_test_text, y_train_marca, y_test_marca, y_train_gatillador, y_test_gatillador = train_test_split(
    image_features_array, text_data, labels_marca_categorical, labels_gatillador_categorical, test_size=0.2, random_state=42)

In [ ]:
with open('/content/drive/MyDrive/archivos_proyecto_deep/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
# Verificar las formas de los datos
print(X_train_image.shape, X_test_image.shape)
print(X_train_text.shape, X_test_text.shape)
print(y_train_marca.shape, y_test_marca.shape)
print(y_train_gatillador.shape, y_test_gatillador.shape)

(3889, 224, 224, 3) (973, 224, 224, 3)
(3889, 100) (973, 100)
(3889, 6) (973, 6)
(3889, 27) (973, 27)


In [ ]:
# Definir el modelo
image_input = Input(shape=(224, 224, 3))
text_input = Input(shape=(max_sequence_length,))

# VGG16 base model
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=image_input)
x = base_model.output
x = GlobalAveragePooling2D()(x)

# RNN for text
embedding = Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length)(text_input)
lstm = LSTM(128)(embedding)

# Concatenate image and text features
concatenated = Concatenate()([x, lstm])
dense = Dense(256, activation='relu')(concatenated)
dropout = Dropout(0.5)(dense)

# Output layers
output_marca = Dense(len(np.unique(labels_marca)), activation='softmax', name='marca_output')(dropout)
output_gatillador = Dense(len(np.unique(labels_gatillador)), activation='softmax', name='gatillador_output')(dropout)
output_gatillador = Dense(len(np.unique(labels_producto)), activation='softmax', name='producto_output')(dropout)
output_gatillador = Dense(len(np.unique(labels_oferta)), activation='softmax', name='oferta_output')(dropout)

# Define model
model = Model(inputs=[image_input, text_input], outputs=[output_marca, output_gatillador])

# Compile model
model.compile(optimizer='adam', loss={'marca_output': 'categorical_crossentropy', 'gatillador_output': 'categorical_crossentropy', 'producto_output': 'categorical_crossentropy', 'oferta_output': 'categorical_crossentropy'}, metrics=['accuracy'])

# Train the model
history = model.fit([X_train_image, X_train_text], {'marca_output': y_train_marca, 'gatillador_output': y_train_gatillador},
                    validation_data=([X_test_image, X_test_text], {'marca_output': y_test_marca, 'gatillador_output': y_test_gatillador}),
                    epochs=10, batch_size=32)

# Guardar el modelo
model.save('/content/drive/MyDrive/archivos_proyecto_deep/multi_output_model.h5')

print("Modelo entrenado y guardado exitosamente!")

58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/10
122/122 [==============================] - 94s 549ms/step - loss: 4.2125 - marca_output_loss: 1.4392 - gatillador_output_loss: 2.7733 - marca_output_accuracy: 0.6680 - gatillador_output_accuracy: 0.4412 - val_loss: 1.9303 - val_marca_output_loss: 0.4060 - val_gatillador_output_loss: 1.5243 - val_marca_output_accuracy: 0.8736 - val_gatillador_output_accuracy: 0.5437
Epoch 2/10
122/122 [==============================] - 54s 446ms/step - loss: 1.5680 - marca_output_loss: 0.3066 - gatillador_output_loss: 1.2615 - marca_output_accuracy: 0.9113 - gatillador_output_accuracy: 0.6169 - val_loss: 1.1901 - val_marca_output_loss: 0.1826 - val_gatillador_output_loss: 1.0075 - val_marca_output_accuracy: 0.9363 - val_gatillador_output_accuracy: 0.6968
Epoch 3/10
122/122 [==============================] - 54s 444ms/step - loss: 1.0590 - marca_output_loss: 0.1893 - gatillador_output_loss: 0.8697 - marca_output_accuracy: 0.9434 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Modelo entrenado y guardado exitosamente!
